In [104]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

config = dotenv_values(".env")
locale.setlocale(locale.LC_ALL, "id")

'id'

In [105]:
url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
obj = {
    'email': config.get('EMAIL'),
    'password': config.get('PASSWORD')
}

response = requests.post(url, json=obj, headers=headers)
token = response.json()['data'].get('access_token')

In [106]:
headers = dict()
headers["Accept"] = "application/json"
headers["Authorization"] = f'Bearer {token}'
headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'

In [107]:
url =  "https://api.kampusmerdeka.kemdikbud.go.id/mbkm/mahasiswa/active-activities/my"
response = requests.get(url, headers=headers)
activity_id = response.json().get('data')[0].get('id')


In [108]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/report/allweeks/{activity_id}'
response = requests.get(url, headers=headers)
reports = response.json().get('data')

In [110]:
weekly_reports = []
daily_reports = []

for report in reversed(list(reports)):
    date = report.get('start_date')[0:10]
    date =  datetime.strptime(date, '%Y-%m-%d')

    date_format = date.strftime("%d %B %Y")
    date_format = date_format.split()

    weekly_reports.append([report.get("learned_weekly"), f"{date_format[1]} {date_format[2]}"])

    for j, daily_report in enumerate(report.get('daily_report')):
        if j == 0:
            dt = date 
        else:
            dt = dt + timedelta(days=1)

        d = dt.strftime("%A, %d %B %Y")
        daily_reports.append([daily_report.get('report'), d])

In [111]:
df_weekly = pd.DataFrame(weekly_reports, columns=['Deskripsi', 'Bulan'])
df_weekly[12:15]

,Deskripsi,Bulan
12,Pada minggu ini masih berfokus untuk menangani...,November 2021
13,Pada minggu ini team backend berfokus untuk me...,November 2021
14,Pada minggu ini team backend berfokus kepada p...,November 2021


In [112]:
df_daily = pd.DataFrame(daily_reports, columns=['Deskripsi', 'Hari'])
df_daily[35:38]

,Deskripsi,Hari
35,"Membuat Endpoint (Terdiri Atas Model, Controll...","Senin, 11 Oktober 2021"
36,Mengerjakan CRUD (Create Read Update Delete) y...,"Selasa, 12 Oktober 2021"
37,Melakukan testing dan evaluasi setiap endpoint...,"Rabu, 13 Oktober 2021"


In [113]:
df_weekly.to_csv('Weekly Report.csv', index=False)
df_daily.to_csv('Daily Report.csv', index=False)